In [1]:
import requests
import pandas as pd
from datetime import datetime,date

In [2]:
def fetch_crypto_data(crypto_pair: str, start_date: str) -> pd.DataFrame:
    """
    Fetches historical cryptocurrency data from the CoinGecko API.
    
    Parameters:
    - crypto_pair (str): The cryptocurrency pair to fetch data for (e.g., "bitcoin/usd").
    - start_date (str): The start date for fetching historical data in 'YYYY-MM-DD' format.
    
    Returns:
    - pd.DataFrame: A DataFrame containing historical price data with columns: Date, Open, High, Low, Close.
    """
    
    # Split the crypto_pair into its components: crypto_id and currency
    crypto_id, currency = crypto_pair.split('/')
    
    # Convert the start date to a UNIX timestamp
    start_timestamp = int(datetime.strptime(start_date, '%Y-%m-%d').timestamp())
    
    # Construct the API URL for fetching market chart data within a specific date range
    url = f"https://api.coingecko.com/api/v3/coins/{crypto_id}/market_chart/range"
    
    # Set up parameters for the API request
    params = {
        'vs_currency': currency.lower(),  # The currency to compare against (e.g., 'usd')
        'from': start_timestamp,           # Start timestamp
        'to': int(datetime.now().timestamp()),  # Current timestamp
    }
    
    # Send a GET request to the CoinGecko API
    response = requests.get(url, params=params)
    
    # Check if the response status is not OK (200)
    if response.status_code != 200:
        raise Exception(f"Error fetching data: {response.status_code} - {response.text}")

    # Parse the JSON response
    data = response.json()

    # Check if 'prices' are present in the response data
    if 'prices' not in data:
        raise Exception("No price data found in the response.")

    # Create a DataFrame from the price data (timestamps and closing prices)
    df = pd.DataFrame(data['prices'], columns=['timestamp', 'close'])
    
    # Convert the timestamp from milliseconds to a date format and add it to the DataFrame
    df['Date'] = pd.to_datetime(df['timestamp'], unit='ms').dt.date
    
    # Initialize Open, High, Low, and Close columns with closing prices
    df['Open'] = df['close']
    df['High'] = df['close']
    df['Low'] = df['close']
    df['Close'] = df['close']

    # Group the DataFrame by date and aggregate the Open, High, Low, and Close values
    df = df.groupby('Date').agg({
        'Open': 'first',  # First entry for Open price
        'High': 'max',    # Maximum price for High
        'Low': 'min',     # Minimum price for Low
        'Close': 'last'   # Last entry for Close price
    }).reset_index()

    return df

# Example usage
if __name__ == "__main__":
    crypto_pair = "bitcoin/usd"  # Use the CoinGecko ID for the cryptocurrency
    start_date = "2024-01-27"
    try:
        crypto_data = fetch_crypto_data(crypto_pair, start_date)
        print(crypto_data)
    except Exception as e:
        print(e)

           Date          Open          High           Low         Close
0    2024-01-27  41862.935611  41862.935611  41862.935611  41862.935611
1    2024-01-28  42119.610495  42119.610495  42119.610495  42119.610495
2    2024-01-29  42026.578562  42026.578562  42026.578562  42026.578562
3    2024-01-30  43267.609276  43267.609276  43267.609276  43267.609276
4    2024-01-31  42892.034530  42892.034530  42892.034530  42892.034530
..          ...           ...           ...           ...           ...
274  2024-10-27  67018.165017  67018.165017  67018.165017  67018.165017
275  2024-10-28  67938.554129  67938.554129  67938.554129  67938.554129
276  2024-10-29  69845.304531  69845.304531  69845.304531  69845.304531
277  2024-10-30  72781.113082  72781.113082  72781.113082  72781.113082
278  2024-10-31  72342.621711  72342.621711  72342.621711  72342.621711

[279 rows x 5 columns]


In [3]:
today = date.today()
print(today)
if(today in crypto_data['Date'].values):
    today_data = crypto_data[crypto_data['Date']==today]
    print(today_data)
else:
    print("today does not exist")

2024-10-31
           Date          Open          High           Low         Close
278  2024-10-31  72342.621711  72342.621711  72342.621711  72342.621711


In [4]:
def calculate_crypto_metrics(df, variable1, variable2):
    # Rolling window calculations for high and low
    df['High_Last_{}_Days'.format(variable1)] = df['High'].rolling(window=variable1, min_periods=1).max()
    df['Low_Last_{}_Days'.format(variable1)] = df['Low'].rolling(window=variable1, min_periods=1).min()

    # Function to get the index of the last maximum or minimum
    def last_max_index(series):
        return series[::-1].idxmax()

    def last_min_index(series):
        return series[::-1].idxmin()

    # Calculate days since last high
    last_high_indices = df['High'].rolling(window=variable1, min_periods=1).apply(last_max_index, raw=False)
    df[f'Days_Since_High_Last_{variable1}_Days'] = (df.index - last_high_indices)

    # Calculate days since last low
    last_low_indices = df['Low'].rolling(window=variable1, min_periods=1).apply(last_min_index, raw=False)
    df[f'Days_Since_Low_Last_{variable1}_Days'] = (df.index - last_low_indices)
    

    #Percentage Difference Calculations
    df['%_Diff_From_High_Last_{}_Days'.format(variable1)] = (
        (df['Close'] - df['High_Last_{}_Days'.format(variable1)]) / df['High_Last_{}_Days'.format(variable1)] * 100
    )

    df['%_Diff_From_Low_Last_{}_Days'.format(variable1)] = (
        (df['Close'] - df['Low_Last_{}_Days'.format(variable1)]) / df['Low_Last_{}_Days'.format(variable1)] * 100
    )

    #Future High and Low Calculations
    df['High_Next_{}_Days'.format(variable2)] = df['High'].shift(-variable2).rolling(window=variable2, min_periods=1).max()
    df['Low_Next_{}_Days'.format(variable2)] = df['Low'].shift(-variable2).rolling(window=variable2, min_periods=1).min()

    #If Nan,then use the previous high value
    df['High_Next_{}_Days'.format(variable2)].fillna(method='ffill', inplace=True)
    df['Low_Next_{}_Days'.format(variable2)].fillna(method='ffill', inplace=True)
    
    # Percentage Difference from Future High and Low
    df['%_Diff_From_High_Next_{}_Days'.format(variable2)] = (
        (df['Close'] - df['High_Next_{}_Days'.format(variable2)]) / df['High_Next_{}_Days'.format(variable2)] * 100
    )

    df['%_Diff_From_Low_Next_{}_Days'.format(variable2)] = (
        (df['Close'] - df['Low_Next_{}_Days'.format(variable2)]) / df['Low_Next_{}_Days'.format(variable2)] * 100
    )
    
    return df

In [5]:
# Set your variable1 and variable2
variable1 = 7
variable2 = 5

# Call the function
df = calculate_crypto_metrics(crypto_data, variable1, variable2)

In [6]:
df

,Date,Open,High,Low,Close,High_Last_7_Days,Low_Last_7_Days,Days_Since_High_Last_7_Days,Days_Since_Low_Last_7_Days,%_Diff_From_High_Last_7_Days,%_Diff_From_Low_Last_7_Days,High_Next_5_Days,Low_Next_5_Days,%_Diff_From_High_Next_5_Days,%_Diff_From_Low_Next_5_Days
0,2024-01-27,41862.935611,41862.935611,41862.935611,41862.935611,41862.935611,41862.935611,0.0,0.0,0.000000,0.000000,42583.263547,42583.263547,-1.691575,-1.691575
1,2024-01-28,42119.610495,42119.610495,42119.610495,42119.610495,42119.610495,41862.935611,0.0,1.0,0.000000,0.613132,43069.043421,42583.263547,-2.204444,-1.088815
2,2024-01-29,42026.578562,42026.578562,42026.578562,42026.578562,42119.610495,41862.935611,1.0,2.0,-0.220876,0.390902,43170.658562,42583.263547,-2.650133,-1.307286
3,2024-01-30,43267.609276,43267.609276,43267.609276,43267.609276,43267.609276,41862.935611,0.0,3.0,0.000000,3.355411,43170.658562,42583.263547,0.224575,1.607077
4,2024-01-31,42892.034530,42892.034530,42892.034530,42892.034530,43267.609276,41862.935611,1.0,4.0,-0.868027,2.458258,43170.658562,42583.263547,-0.645401,0.725099
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274,2024-10-27,67018.165017,67018.165017,67018.165017,67018.165017,68962.829180,66585.665355,6.0,1.0,-2.819873,0.649539,72781.113082,67938.554129,-7.918192,-1.354738
275,2024-10-28,67938.554129,67938.554129,67938.554129,67938.554129,68214.052052,66585.665355,3.0,2.0,-0.403873,2.031802,72781.113082,69845.304531,-6.653593,-2.729962
276,2024-10-29,69845.304531,69845.304531,69845.304531,69845.304531,69845.304531,66585.665355,0.0,3.0,0.000000,4.895407,72781.113082,72342.621711,-4.033751,-3.452069
277,2024-10-30,72781.113082,72781.113082,72781.113082,72781.113082,72781.113082,66585.665355,0.0,4.0,0.000000,9.304477,72342.621711,72342.621711,0.606131,0.606131


In [7]:
df.to_excel('crypto.xlsx',sheet_name='CryptoData',index=False)